In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd


service = Service(ChromeDriverManager().install())

options = Options()
options.add_argument("--headless")
options.add_argument("--window-size=1920x1080")
options.add_argument("--lang=ko_KR")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0")

driver = webdriver.Chrome(service=service, options=options)


url = "https://display.wconcept.co.kr/rn/best?displayCategoryType=10102&displaySubCategoryType=ALL&gnbType=Y"
driver.get(url)
time.sleep(5)

products = driver.find_elements(By.CSS_SELECTOR, "span.sc-21e95a6c-0.hFtgIp.text.detail")
ratings = driver.find_elements(By.CSS_SELECTOR, "em.score")
reviews = driver.find_elements(By.CSS_SELECTOR, "span.sc-21e95a6c-0.hFtgIp.text.cnt")

print(len(products), len(ratings), len(reviews))

data = []

for pr, ra, re in zip(products[:30],ratings[:30],reviews[:30]) :
    data.append({
        "product" : pr.text.strip(),
        "rating" : ra.text.strip(),
        "review" : re.text.strip()
    })

driver.quit()

df = pd.DataFrame(data)

print(df.head())
print(df.info())

200 175 375
                                 product rating   review
0  [한정특가] Veneta Hobo Bag Medium_7Colors    4.9    (329)
1          Sac de Flot Large FA4SB269-FH    4.9   9,999+
2  [14%쿠폰] [단독] Panier Petit Bag_6colors    5.0    (711)
3  [14%쿠폰] [단독] Panier Grand Bag_2colors    4.9   9,999+
4     [단독특가] Rabbit eco bag_black flower    4.9  (3,721)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   product  30 non-null     object
 1   rating   30 non-null     object
 2   review   30 non-null     object
dtypes: object(3)
memory usage: 852.0+ bytes
None


In [21]:
import pymysql

conn = pymysql.connect(
    host = 'localhost', 
    user = 'root',
    password = '753951DnDn!!',
    db = 'wconcept_soobin',
    charset = 'utf8mb4'
)

cursor = conn.cursor()

for i, row in df.iterrows():
    sql = """
    INSERT INTO performances (product, rating, review)
    VALUES (%s, %s, %s)
    """
    cursor.execute(sql, (row["product"], row["rating"], row["review"]))

conn.commit()
conn.close()


OperationalError: (1049, "Unknown database 'wconcept_soobin'")